# Clase Práctica 1: Clasificación 📈


----------------------------------

En el aprendizaje de máquinas, la tarea de clasificación consiste en predecir una variable discreta (perteneciente a un conjunto finito de opciones) a partir de una observación, es decir, un conjunto de características. 

La principal diferencia entre problemas de clasificación y regresión, es que en estos últimos los modelos predicen una variable continua (conjunto infinito de posibilidades) a partir de los datos entrada. Así, un ejemplo de un problema de regresión sería predecir el precio de una casa a partir de las características como el tamaño, ubicación, material, etc. De lo contrario, un ejemplo de clasificación sería predecir si el contenido de un correo corresponde a un spam o no. En este caso, como el número de respuestas posibles es igual a 2 (finito), se dice que es un problema de clasificación binaria. 

## Objetivos de la clase 📚

El objetivo principal de esta primera clase práctica es realizar una introducción a la tarea de clasificación en el aprendizaje de máquinas. Para esto, primero implementaremos varios modelos de clasificación para poder relacionar las características fisiológicas de pacientes con la presencia de una enfermedad cardiovascular 🩺. 

Las principales librerías que vamos a utilizar son las siguientes:

- Pandas 🐼: Manejo y análisis de estructuras de datos.
- Scikit-learn: Librería para el aprendizaje de máquina, contiene no sólo modelos clásicos sino que también una serie de funciones que nos permitirán procesar mejor los datos de entrada.
- Numpy: Manejo de operaciones matemáticas.
- Matplotlib y Seaborn: Generación de visualizaciones sobre los datos.

## Metodología

La mayoría de los problemas que se resuelven con aprendizaje de máquinas pueden ser abordados siguiendo 3 pasos principales:

1. Exploración de datos.
2. Selección de modelos.
3. Comparación y análisis de resultados.

A pesar de que en aplicaciones reales existen muchos más pasos intermedios, como el estudio del problema a nivel de negocio y la puesta en producción, estos pasos serán suficientes para abordar los distintos problemas que veremos en este curso.

Así que empecemos! 🥳


## **Clasificación**

**Exploración de datos 🧐**

En la mayoría de los casos, cuando se resuelven problemas con aprendizaje de máquinas solemos trabajar con datos almacenados en formatos de tablas, con extensiones tales como *csv*, *tsv*, *xlsx* o incluso en algunos casos *txt* (más común al trabajar con texto). 

*Pandas* 🐼 es una librería de Python especializada en el manejo de estructuras de datos, la cual nos permite poder cargar, procesar y analizar datos tabulados de manera sencilla. Además, nos entrega muchas funciones que en conjunto con otros librerías de visualización como *Matplotlib* o *Seaborn* nos permite obtener excelentes visualizaciones sobre los datos.

**Importar las librerías**

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

**Lectura del dataset**

Con la función `read_csv` podemos cargar datasets que se encuentren en formato csv. Es importante que el nombre de la variable sea lo más simple posible, ya que será utilizado frecuentemente.

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/fvillena/biocompu/2022/data/cardiovascular_diseases.csv")

La función `head` nos muestra las primeras 5 filas del dataset

In [ ]:
df.head()

La función `sample(x)` nos entrega x ejemplos del datasets al azar

In [ ]:
df.sample(3)

La función `tail` nos muestra las últimas filas del dataset

In [ ]:
df.tail()

En aprendizaje de máquinas las filas de un dataset son más conocidas como *instancias*, mientras que las columnas son nuestros *atributos* o *features*. Para acceder al número de instancias y de features utilizamos el atributo `shape`. El primer valor corresponde al número de instancia, mientras que el segundo es el número de atributos.

In [ ]:
df.shape 

El resultado de la celda anterior nos muestra que hay un total de 301 instancias, donde cada una tiene un total de 14 atributos. Para acceder al nombre de cada uno de estos atributos, utilizamos el comando `columns`.

In [ ]:
df.columns

Según el resultado anterior, tenemos los siguientes datos del paciente: edad, sexo, tipo de dolor de pecho, colesterol, nivel alto de azucar en sangre en ayudo, resultados electrocardiográficos en reposo, frecuencia cardiaca máxica alcanzada, angina inducida por el ejercicio, depresión st por ejercicio, entre otras.

Si queremos información más detallada acerca de estos atributos podemos utilizar la función `info`, la cual nos entrega el nombre, la cantidad de ejemplos no nulos y el tipo de dato de cada columna.

In [ ]:
df.info()

Si queremos obtener estadística acerca de cada una de nuestras columnas utilizamos la función `describe`. Esta función nos entrega la cantidad de datos, el promedio, desviación estandar, mínimo, máximo y cuartiles.

In [ ]:
df.describe()

Un paso fundamental en la exploración de datos es ver que no hayan valores nulos. Para revisar esto de manera rápida, utilizamos la función `isnull`. Esta función analiza celda por celda si existe un valor nulo, en caso que así sea asigna el valor `True`, en caso contrario se coloca el valor `False`.

In [ ]:
df.isnull()

Luego, podemos utilizar la función `sum`, que nos permite sumar los valores que son `True` en cada columna.

In [ ]:
df.isnull().sum()

En este caso no tenemos valores nulos, pero si los tuviéramos es importante que sean reemplazados por algún valor. 

Ahora vamos a analizar nuestro **atributo objetivo**. Este atributo corresponde al valor que nos gustaría predecir a partir de los demás atributos (o parte de ellos). En nuestro caso utilizaremos la columna  *cardiovascular_disease*. Accedamos a ella para ver la naturaleza de los datos.

In [ ]:
df["cardiovascular_disease"]

Al parecer se trata de un atributo con valores 0's y 1's, donde 0 correspondería a que el paciente no presenta enfermedad cardiovascular, mientras que 1 significa que si tiene. Para comprobar esto, utilizamos la función `set` que nos permite ver todos los valores distintos que puede tomar este atributo.

In [ ]:
set(df["cardiovascular_disease"])

Con este resultados podemos concluir que trabajaremos con una clasificación binaria 👀. Otro paso fundamental, es contar la frecuencia de cada uno de los posibles valores pertenecientes a nuestro atributo/variable objetivo. Esto se realiza con la función `value_counts`.

In [ ]:
df["cardiovascular_disease"].value_counts()

Podemos ver que los datos están prácticamente balanceados; 164 pacientes presentan una enfermedad cardiovascular, mientras que 137 no tienen. En caso que exista un desbalance, hay que aplicar técnicas especiales de balanceo que veremos más adelante en el curso. Ahora sigamos analizando nuestros datos viendo un ejemplo.

In [ ]:
df.sample(1)

Podemos ver que las columnas *chest_pain_type* y *thalassemia* son atributos con valores de tipo string (palabras). En aprendizaje de máquinas nuestros algoritmos o sistemas están basados en una serie de operaciones matemáticas sobre los datos, lo cual nos permite entrenar y luego realizar predicciones. Debido a esto, los modelos no pueden trabajar directamente con texto y estos deben ser transformados a números.

La manera más sencilla de realizar esto es utilizando una transformación **One-Hot**. Esto significa que si tenemos un atributo de tipo string, el cual tiene `n` valores distintos, crearemos un total de `n` columnas nuevas con valores numéricos y binarios. Para ejemplificar, si se tiene una columna con los siguientes valores


| chest_pain_type |
|--------|
| asymptomatic |
| non_anginal_pain   |
| typical_angina|
| atypical_angina |

y se transforma utilizando One-Hot Encoding, se generan las siguientes asignaciones columnas:


| chest_pain_type_asymptomatic | chest_pain_type_non_anginal_pain | chest_pain_type_typical_angina | chest_pain_type_atypical_angina |
|------------------------------|----------------------------------|--------------------------------|---------------------------------|
| 1                            | 0                                | 0                              | 0                               |
| 0                            | 1                                | 0                              | 0                               |
| 0                            | 0                                | 1                              | 0                               |
| 0                            | 0                                | 0                              | 1                               |


Así, ya no tenemos que lidiar con valores no numéricos, y la cantidad de características aumenta. Para realizar esta transformación podemos utilizar la función `get_dummies`.

In [ ]:
df = pd.get_dummies(df)
df.sample(3)

Un dato que puede ser útil a la hora de analizar datasets, es que los dataframes de pandas nos permiten realizar consultas sobre nuestro dataset con los filtros. Por ejemplo, con el siguiente comando consultamos cuántas personas menores a 40 años presentan una enfermedad cardiovascular.

In [ ]:
df[(df["age"] <= 40) & (df["cardiovascular_disease"] == 1)].shape[0]

Lo siguiente que podemos hacer, es crear diferentes visualizaciones que nos permitan entender mejor nuestros datos. 

Por ejemplo, si utilizamos la función *box plot* podemos visualizar si es que existen datos alejados del promedio o con un extraño comportamiento en nuestro dataset, más conocidos como outliers.

In [ ]:
df[['age', 'cholesterol','resting_blood_pressure']].plot.box(vert = False, grid = True)

También podemos utilizar un scatter plot para ver si dos variables tienen una relación lineal o algún otro comportamiento

In [ ]:
plt.scatter(df['age'], df['cardiovascular_disease'])
plt.xlabel('Edad')
plt.ylabel('Enfermedad Cardiovascular')
plt.show()

Con la función `corr` visualizamos la matriz de correlación entre nuestros atributos

In [ ]:
corr = df.corr()
features = corr.index
plt.figure(figsize=(15,15))
ax = sns.heatmap(df[features].corr(), annot=True)

**Entrenamiento de modelos**

Para poder entrenar nuestros modelos de aprendizaje supervisado debemos separar nuestros datos identificando claramente cuál será nuestra variable objetivo, es decir, la que queremos predecir. 

Generalmente se utiliza la variable `X` para los atributos de entrada o features, mientras que la variable `y` es el objetivo.

In [ ]:
X = df.drop("cardiovascular_disease", axis=1) # Drop elimina la columna que se especifique, y axis = 1 significa eliminar valores de las columnas

In [ ]:
y = df['cardiovascular_disease']

Teniendo ambos vectores, ya podemos entrenar modelos sobre nuestros datos 😄. En particular, veremos 3 modelos: Support Vector Machines (SVM), Decision Tree Classifier (DTC), y Randon Forest (RF).


**Support Vector Classification (SVC)**



Una explicación muy general sobre este modelo, es que representa a los puntos de muestra en el espacio, separando las clases a 2 espacios lo más amplios posibles mediante un hiperplano de separación. Cuando llega un nuevo ejemplo, se analiza a cuál de los dos espacios pertenece para realizar la clasificación. En general, este algoritmo funciona muy bien con datos que son linealmente separables. En caso que no sea así, este algoritmo puede trasladar el problema de un problema no separable linealmente a uno que sí lo es. Veamos como podemos programar un modelo SVC.

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_model = SVC() # Creamos un objeto de tipo SVC.
svc_model.fit(X, y) # Entrenamos el modelo con nuestros vectores X e y

Luego de que el modelo fue entrenado, podemos hacer  predicciones sobre algún conjunto de datos para así analizar el rendimiento de nuestro clasificador. Este proceso se realiza mediante la función `predict`.

In [ ]:
predictions = svc_model.predict(X)

Para evaluar el rendimiento de nuestro modelo utilizamos la función `accuracy_score`. Esta función nos permite obtener la proporción de ejemplos que fueron correctamente clasificados sobre el total.

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
print(f'El accuracy obtenido por el modelo SVC es: {np.round(accuracy_score(y, predictions), 2)}')

Y listo, hemos entrenado nuestro clasificador y hemos obtenido resultados bastante altos dada la complejidad del problema 🥳 



![Fired!](https://media3.giphy.com/media/3EAYL7KCtZJOJGtli6/giphy.gif?cid=ecf05e47b8ucx71fgr2bny9bn1mf8osjbh2z2grz4c4v4pyd&rid=giphy.gif&ct=g)

Sin embargo, hemos entrenamos el modelo sobre el conjunto de datos `X` e `y` y luego medimos el rendimiento sobre el mismo conjunto `y` original, es decir, no estamos midiendo correctamente el rendimiento del modelo en datos nuevos. Es como si llegaran 10 pacientes a la consulta, aprendemos cuál de ellos tiene o no tiene una enfermedad vascular. Y luego, evaluamos que tan bien clasificamos a los mismos 10 pacientes si tienen o no tienen la enfermedad. Esto no sólo es hacer trampa, sino que nos lleva al problema de **generalización** en aprendizaje de máquinas.

¿Qué es la generalización?

Supongamos que ustedes son unos estudiantes de primer año de universidad. Para poder estudiar para su primer control recolectan los controles realizados en toda la historia del ramo, y con una gran memoria se aprenden de memoria todas las preguntas y respuestas. 

Si al momento de rendir la prueba, aparece alguna de esas mismas preguntas, ustedes obtendrán una buena nota ya que saben la respuesta. Sin embargo, si por alguna razón el profesor decide innovar y crea una nueva pregunta, la nota que obtendrán probablemente será mala, ya que no estaban preparados para una nueva pregunta, un nuevo dato. Eso es lo que ocurre acá, no sabemos si el modelo es capaz de obtener buenos resultados en nuevos datos. 

¿Cómo podemos solucionar esto?

Es importante antes de entrenar un modelo dividir nuestros datos en dos conjuntos; entrenamiento y testeo. El primer conjunto nos servirá para entrenar nuestro modelo, mientras que el segundo nos servirá para medir el rendimiento en datos que con alta probabilidad no fueron vistos anteriormente. Esto nos permite medir el nivel de **generalización** de los modelos.

Más adelante veremos que en la práctica es mejor dividir el conjunto original en 3 particiones; entrenamiento, validación y testeo. Pero por ahora, nos quedaremos con las dos mencionadas. Para generar estos conjuntos lo hacemos con el siguiente código:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=123)

Luego, creamos y entrenamos nuestro modelo sobre el conjunto de entrenamiento.

In [ ]:
svc_model = SVC()
svc_model.fit(X_train, Y_train)

Obtenemos las predicciones sobre el conjunto de test y calculamos el rendimiento.

In [ ]:
predictions = svc_model.predict(X_test)

In [ ]:
print(f'El accuracy obtenido por el modelo SVC es: {np.round(accuracy_score(Y_test, predictions), 2)}')

In [ ]:
# Podemos obtener los indices de los datos que nos permiten crear el hiperplano
support_vector_indices = svc_model.support_
print(support_vector_indices)

Dado que estamos trabajando el múltiples dimensiones no es posible generar un gráfico adecuado, sin embargo, si tuvieramos un problema de clasificación binaria con sólo dos atributos de entrada, podríamos generar visualizaciones como la siguiente. Este método es diferente de una Regresión logistica, ya que busca encontrar la máxima separación entre los conjuntos, no una sóla recta.

![image info](https://github.com/christianversloot/machine-learning-articles/raw/main/images/support_vectors.png)

**Decission Tree Classifier**

Método de aprendizaje supervisado que permite resolver tareas de regresión y clasificación. Se llama árbol ya que visualmente parece un árbol invertido, y está compuesto tanto por una raíz, ramas y hojas como veremos más adelante. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
tree = DecisionTreeClassifier( # Instanciamos nuestro árbol de decisión.
    max_depth=3, # Forzamos que nuestro árbol sólo tenga 3 niveles de profundidad.
    random_state = 11
    )
tree.fit(X_train, Y_train)

In [ ]:
tree_predictions = tree.predict(X_test)

In [ ]:
print(f'El accuracy obtenido por el modelo Decision Tree es: {np.round(accuracy_score(Y_test, tree_predictions), 2)}')

Una de las ventajas más importantes de los árboles de decisión es la transparencia y explicabilidad del proceso de predicción. Podemos conocer claramente todas las decisiones tomadas por el clasificador y visualizarla en un árbol de decisión.

In [ ]:
plt.figure(figsize = (20,10))
plot_tree( # Función que nos permite visualizar el árbol de decisión ajustado.
    tree,
    class_names = ["healthy","sick"],
    feature_names = df.columns,
    filled=True,
    fontsize=11,
    proportion = True,
    label = "all"
    
    
    )
plt.show()

Como se ve en la figura anterior, este método evalua todas las variables de entrada para seleccionar la mejor salida. La variable más significativa se encontrará en la base del árbol creando una condición, a partir de esto se generan n posibilidades dependiendo la cantidad de posibles valores de la variable objetivo, luego se pasa al siguiente nodo y se repite el proceso, hasta llegar a una hoja.

Para evaluar el nivel de importancia que tienen las caracterñisticas, los algoritmos basados en árboles nos ofrecen la importancia de cada variable al calcular normalmente el Mean Decrease Gini, pueden encontrar más información acá https://medium.com/the-artificial-impostor/feature-importance-measures-for-tree-models-part-i-47f187c1a2c3

In [ ]:
tree_vil = pd.DataFrame(list(zip(df.columns,tree.feature_importances_)),
             columns=["feature","importance"]
            ).set_index("feature")
tree_vil.sort_values("importance",ascending=False)

Uno de los hiperparámetros de los árboles de decisión es la profundidad del mismo. Evaluemos cómo se comporta su rendimiento al cambiar la profundidad del árbol.

In [ ]:
depths = range(1,10)
performances = []
for depth in depths:
    current_tree = DecisionTreeClassifier( # Instanciamos nuestro árbol de decisión.
    max_depth=depth, # Forzamos que nuestro árbol sólo tenga 3 niveles de profundidad.
    random_state = 11
    )
    current_tree.fit(X_train, Y_train)
    predictions = current_tree.predict(X_test)

    performances.append(accuracy_score(Y_test, predictions))

In [ ]:
plt.plot(
    depths,
    performances
)
plt.xlabel("max_width")
plt.ylabel("mean_roc_auc_score")
plt.show()

**Random Forest Classifier**

Método basado en el modelo anterior. La idea es que en vez de tener un único árbol, existirán múltiples, de ahí viene el término forest. Dado que cada árbol entrega una clasificación, la clasificación quedará determinada por el valor que contenga más votos. Entre más árboles haya, más robusto y preciso será el algoritmo.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

In [ ]:
rf_predictions = rf.predict(X_test)

In [ ]:
accuracy_score(Y_test, rf_predictions)

In [ ]:
rf_vil = pd.DataFrame(list(zip(df.columns,rf.feature_importances_)),
             columns=["feature","importance"]
            ).set_index("feature")
rf_vil.sort_values("importance",ascending=False)

In [ ]:
random_tree = rf.estimators_[-3]

In [ ]:
plt.figure(figsize = (20,10))
plot_tree( # Función que nos permite visualizar el árbol de decisión ajustado.
    random_tree, # Objeto de nuestro árbol de decisión entrenado.
    feature_names = df.columns, # Nombres de las variables utilizadas para entrenar.
    class_names = ["healthy","sick"], # Nombre de las clases que estamos prediciendo.
    label = "all", # Etiquetamos todas características de cada nodo.
    proportion = True, # Visualizamos las proporciones de datos en cada nodo de decisión,
    filled=True, # Coloreamos los nodos
    fontsize=11, # Establecemos el tamaño de la letra del texto dentro de cada nodo.,
    max_depth=3 # Profundidad máxima del árbol
)
plt.show()